In [1]:
import pandas as pd

In [2]:
import datetime
import os

In [3]:
def date_genetator(starting_date):
    date=datetime.datetime.strptime(starting_date,"%d.%m.%Y")
    while True:
        yield date
        date-=datetime.timedelta(days=7)

In [4]:
data_dir="../NewsParsing"
publishers=["kommersant","lenta","interfax","gazeta","rain"]
starting_date="09.11.2020"

In [5]:
def build_data():
    data=[]
    for date in date_genetator(starting_date):
        tmp=[]
        for pub in publishers:
            try:
                fie=os.path.join(data_dir,pub,datetime.datetime.strftime(date,"%Y-%m-%d.csv"))
                tmp.append(pd.read_csv(fie, encoding='utf-8'))
                tmp[-1]["publisher"]=pub
            except:
                print("Real end date:", datetime.datetime.strftime(date,"%Y-%m-%d"))
                return data
        data+=tmp

In [6]:
raw_data=build_data()

Real end date: 2015-12-28


In [7]:
data=pd.concat(raw_data, ignore_index=True)
data=data[data["text"].notna() & data["title"].notna()]

In [8]:
data["id"]=data["publisher"]+"_"+data["url"].apply(lambda x: x.rstrip('/').split('.ru/',1)[1])

In [9]:
data["fdate"]=data["date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())

In [24]:
import json

In [29]:
sh_data=data.sample(frac=1,random_state=100)

In [33]:
from math import ceil

In [34]:
val_thr=ceil(len(sh_data)*0.8)
test_thr=ceil(len(sh_data)*0.9)

In [35]:
val_thr

806852

In [40]:
names=['train','validate','test']
fd="./headline_dataset"

writers=[open(os.path.join(fd,n+'.jsonl'),encoding='utf-8',mode='w') for n in names]

for idx, row in sh_data.iterrows():
    split=(idx>val_thr)+(idx>test_thr)
    tow={'target':row['title'],
         'source':row['text'].replace('\\n',' ')}
    json.dump(tow,writers[split], ensure_ascii=False)
    writers[split].write('\n')

for w in writers:
    w.close()

In [66]:
tow

{'target': 'СКР возбудил дело по статье «Убийство» после трагедии с псковскими подростками',
 'source': 'Следственное управление СКР по Псковской области возбудило дело по ст. 105 УК РФ (убийство) в отношении неустановленного лица после того, как в частном доме поселка Струги Красные были обнаружены тела 15-летних юноши и девушки. Официальный представитель ведомства Юрий Зайцев отметил, что «до установления всех обстоятельств преступления дело будет расследоваться по этой статье». «Если в рамках расследования будет установлено, что произошло самоубийство, дело будет закрыто»,— приводит ТАСС его слова. Напомним, 14 ноября в поселке Струги Красные 15-летние юноша и девушка забаррикадировались в частном доме и обстреливали полицейских из охотничьего ружья. В течение нескольких часов подростки вели прямые видеотрансляции происходящего в осажденном бойцами СОБРа здании в социальных сетях. После штурма в доме были обнаружены тела школьников с огнестрельными ранами. По официальному заявлению 

In [45]:
from razdel import sentenize, tokenize

In [71]:
names=['train','validate','test']
fd="./headline_dataset"
td="./headline_parts"
size=2000
pn='ruhead'

for n in names:
    with open(os.path.join(fd,n+'.jsonl'),encoding='utf-8') as f:
        buf=[]
        for idx, line in enumerate(f):
            if idx>0 and idx%size==0:
                with open(os.path.join(td,pn+'.'+n+'.'+str(idx//size-1)+'.json'), encoding='utf-8', mode='w') as pf:
                          json.dump(buf, pf, ensure_ascii=False)
                          buf=[]
            tmp=json.loads(line, encoding='utf-8')
            buf.append({
                'tgt':[[i.text for i in tokenize(j.text)] for j in sentenize(tmp['target'])],
                'src':[[i.text for i in tokenize(j.text)] for j in sentenize(tmp['source'])]
            })